# HW01 Solutions


## Q1.


a) When a slight reward adjustment of \(c\) is introduced to all transitions in the MDP, the new optimal value function, denoted as $(V_{1,c})$, can be expressed as:

${ V_{1,c}(s) = c \cdot \sum_{s'} P(s' \,|\, s, \pi_1(s))[R(s, \pi_1(s), s') + \gamma \cdot V_{1,c}(s')] }$

The optimal policy ${\pi_1}$ may or may not change, depending on the specific nature of the MDP and the magnitude and sign of the reward adjustment c.

b) If all rewards are scaled by an arbitrary real constant $c \in \mathbb{R}$, the new optimal value function $V_{1,c}$ can be expressed as:

$ V\_{1,c}(s) = c \cdot V_1(s) $

The optimal policy $\pi_1$ remains unchanged when scaling rewards, as the policy is determined by the relative values and remains optimal for the scaled rewards. Therefore, $\pi_1$ remains optimal for any choice of $c$.

c) The presence of terminal states does affect the response to part (a). If there are terminal states that end an episode, the response to part (a) changes. The introduction of terminal states creates a finite horizon problem, and the optimal value function becomes time-dependent. The new optimal value function is denoted as $V_{1,c,T}$, and it satisfies the following expression:

$ V*{1,c,T}(s) = c \cdot \sum*{s'} P(s' \,|\, s, \pi*1(s))[R(s, \pi_1(s), s') + \gamma \cdot V*{1,c,T}(s')], \quad s \notin T $

where $T$ is the set of terminal states.

An example where responses to part (a) and part (c) would diverge is an MDP with terminal states where the optimal policy for the original MDP $\pi_1$ might not be optimal in the modified MDP with terminal states.


## Q2 Pacman


### import libraries


In [1]:
import numpy as np
import random
import pygame
import time

pygame 2.6.1 (SDL 2.28.4, Python 3.9.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


### Display class for optional section:


In [8]:
class display:
    def __init__(self) -> None:
        pass
    
    def Animation(board):
        WIDTH, HEIGHT = 60, 50
        map_data = np.array(board)
        window_size = (WIDTH * map_data.shape[1], HEIGHT * map_data.shape[0])
        screen = pygame.display.set_mode(window_size)
        screen.fill((0, 0, 0))
        WHITE = (255, 255, 255)
        BRWON = (150, 75, 0)
        GREEN = (0, 128, 0)
        
        for y, row in enumerate(map_data):
            for x, cell in enumerate(row):
                rect = pygame.Rect(x * WIDTH, y * HEIGHT, WIDTH, HEIGHT)
                if cell == 'W':
                    pygame.draw.rect(screen, BRWON, rect)
                elif cell == 'A':
                    pygame.draw.rect(screen, GREEN, rect)
                elif cell == 'D':
                    pygame.draw.circle(screen, WHITE, (x * WIDTH + WIDTH // 2, y * HEIGHT + HEIGHT // 2), 10)
        pygame.display.flip()
        pygame.display.update()
        pygame.time.delay(100)

In [3]:
class Board:
    def __init__(self) -> None:
        self.initilaBoard = [
            ['W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W'],
            ['W', 'A', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'W'],
            ['W', 'D', 'W', 'W', 'W', 'D', 'W', 'W', 'W', 'D', 'W'],
            ['W', 'D', 'W', 'D', 'D', 'D', 'D', 'D', 'W', 'D', 'W'],
            ['W', 'D', 'D', 'D', 'W', 'E', 'W', 'D', 'D', 'D', 'W'],
            ['W', 'D', 'W', 'D', 'W', 'E', 'W', 'D', 'W', 'D', 'W'],
            ['W', 'D', 'W', 'D', 'D', 'W', 'D', 'D', 'W', 'D', 'W'],
            ['W', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'W'],
            ['W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W']
        ]
        self.qTable = np.zeros((3**8, 4)) 
        self.currentBoard = []

    def makeNewBoard(self) -> list:
        return [i.copy() for i in self.initilaBoard]   

    def getBoardRowsNumber()->int:
        return 7

    def getBoardColumnsNumber()->int:
        return 9

In [4]:
class QLearning(Board):
    
    alpha = 0.7
    gamma = 0.8
    epcilon = 0.9
    
    WALL_REWARD = -2
    EMPTHY_REWARD = -1
    DOT_REWARD = 2
    REWARD_MAPPER = {
        "W" : WALL_REWARD , 
        "E" : EMPTHY_REWARD, 
        "D" : DOT_REWARD,
    }

    HASH_MAPPER = {
        "W" : 0, 
        "E" : 1, 
        "D" : 2
    }

    ACTION_MAPPER = {
        0 : "UP", 
        1 : "DOWN", 
        2: "RIGHT", 
        3:"LEFT",
    }


    def __init__(self) -> None:
        self.preResult = 0
        super().__init__()

    def QFunction(self,q :int, qMax:int, reward:int):
        return q + self.alpha * (reward + self.gamma * qMax - q)

    def qTableHash(self, i:int , j:int)->int:
        iter = 0
        result = 0
        for m in range(-1, 2):
            for n in range(-1, 2):
                if m!= 0 or n !=0:
                    result += (self.HASH_MAPPER[self.currentBoard[i+m][j+n]] * (3**iter))
                    iter += 1
        return result

    def getStateAndAction(self, i: int, j: int):
        if random.random() < self.epcilon:
            return self.qTableHash(i, j) ,random.randint(0,3)
        return self.qTableHash(i, j), np.argmax(self.qTable[self.qTableHash(i, j)])
    
    def testGetStateAndAction(self, i:int, j:int): 
        if (self.qTableHash(i,j) == self.preResult): #huristic
            tmp = []
            if(self.currentBoard[i+1][j]!="W") : tmp.append(1)
            if(self.currentBoard[i-1][j]!="W") : tmp.append(0)
            if(self.currentBoard[i][j+1]!="W") : tmp.append(2)
            if(self.currentBoard[i][j-1]!="W") : tmp.append(3)
            return self.qTableHash(i, j), random.choice(tmp)
        self.preResult =  self.qTableHash(i, j)
        return self.qTableHash(i, j), np.argmax(self.qTable[self.qTableHash(i, j)])

In [5]:
class PackMan(QLearning):
    def __init__(self) -> None:
        super().__init__()

    def resetActionPosition(self)->None:
        self.i= 1
        self.j = 1 

    def calculateNumberOfDots(self)->int:
        return sum(row.count('D') for row in self.initilaBoard) 

    def run(self, mode = "train", displayAnime = False):
        self.resetActionPosition()
        self.numberOfDots = self.calculateNumberOfDots()

        if(mode == "test" and displayAnime):
            pygame.init()
            display.Animation(self.currentBoard)

        while self.numberOfDots > 0 :
        
            if mode == "train":
                currentState, action = self.getStateAndAction(self.i, self.j)
            else:
                currentState, action = self.testGetStateAndAction(self.i, self.j)  
            reward = 0
            
            if self.ACTION_MAPPER[action] == "UP":
                if self.currentBoard[self.i-1][self.j] != "W":
                    reward = self.REWARD_MAPPER[self.currentBoard[self.i-1][self.j]]
                    self.currentBoard[self.i-1][self.j] = "A"
                    self.currentBoard[self.i][self.j] = "E"
                    self.i -=1    
            elif self.ACTION_MAPPER[action] == "DOWN":
                if self.currentBoard[self.i+1][self.j] != "W":
                    reward = self.REWARD_MAPPER[self.currentBoard[self.i+1][self.j]]
                    self.currentBoard[self.i+1][self.j] = "A"
                    self.currentBoard[self.i][self.j] = "E"
                    self.i +=1         
            elif self.ACTION_MAPPER[action] == "RIGHT":
                if self.currentBoard[self.i][self.j+1] != "W":
                    reward = self.REWARD_MAPPER[self.currentBoard[self.i][self.j+1]]
                    self.currentBoard[self.i][self.j+1] = "A"
                    self.currentBoard[self.i][self.j] = "E"
                    self.j +=1 
            elif self.ACTION_MAPPER[action] == "LEFT":
                if self.currentBoard[self.i][self.j-1] != "W":
                    reward = self.REWARD_MAPPER[self.currentBoard[self.i][self.j-1]]
                    self.currentBoard[self.i][self.j-1] = "A"
                    self.currentBoard[self.i][self.j] = "E"
                    self.j -=1

            if reward == self.DOT_REWARD:
                self.numberOfDots -=1

            nextState = self.qTableHash(self.i, self.j)
            if mode == "train":
                self.qTable[currentState][action] = self.QFunction(self.qTable[currentState][action], max(self.qTable[nextState]), reward)
                
            if mode == "test" and displayAnime: 
                display.Animation(self.currentBoard)
                       
    def setAndChangeParams(self, alpha, gamma, epsilon)->None:
        self.alpha = alpha
        self.gamma = gamma
        self.epcilon = epsilon
        
    def train(self):
        n = 80
        for i in range(n):
            self.currentBoard = self.makeNewBoard() 
            if i < 40:
                self.epcilon = self.epcilon * ((n-(i%25))/ n)
            self.run()  
        print("done")
        
    def test(self, displayAnime = True):
        self.resetActionPosition()
        self.currentBoard = self.makeNewBoard()
        self.run("test", displayAnime)
        print(np.array(self.currentBoard))
        pygame.quit()

In [6]:
packMan = PackMan()
packMan.train()

done


In [10]:
packMan.test()

[['W' 'W' 'W' 'W' 'W' 'W' 'W' 'W' 'W' 'W' 'W']
 ['W' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'W']
 ['W' 'E' 'W' 'W' 'W' 'E' 'W' 'W' 'W' 'E' 'W']
 ['W' 'E' 'W' 'E' 'E' 'E' 'E' 'E' 'W' 'E' 'W']
 ['W' 'E' 'E' 'E' 'W' 'E' 'W' 'E' 'E' 'E' 'W']
 ['W' 'E' 'W' 'A' 'W' 'E' 'W' 'E' 'W' 'E' 'W']
 ['W' 'E' 'W' 'E' 'E' 'W' 'E' 'E' 'W' 'E' 'W']
 ['W' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'W']
 ['W' 'W' 'W' 'W' 'W' 'W' 'W' 'W' 'W' 'W' 'W']]


#### 2.1


please check pdf file.


##### 2.2


As you can see in my code, I definde my action like this:

```py
ACTION_MAPPER = {
    0 : "UP",
    1 : "DOWN",
    2: "RIGHT",
    3:"LEFT",
}
```

and as I said in the first question I have a hash function that defined my states, every state is hash of 8 adjacent element of that.
and as you can see in code my rewards are:

```py
WALL_REWARD = -2
EMPTHY_REWARD = -1
DOT_REWARD = 2
REWARD_MAPPER = {
    "W" : WALL_REWARD ,
    "E" : EMPTHY_REWARD,
    "D" : DOT_REWARD,
}
```

and finally my goal is to decrease dots, and when number of dots reached to 0 I will terminate the game.


#### 2.3


In [8]:
for gamma in [0.25, 0.5, 0.99]:
    packMan = PackMan()
    packMan.setAndChangeParams(packMan.alpha, gamma, 0.9)
    start = time.time()
    packMan.train()
    end = time.time()
    print("for gamma = " , gamma , " train duration time : ", end - start)
    start = time.time()
    packMan.test(False)
    end = time.time()
    print("for gamma = " , gamma , " test duration time : ", end - start)

done
for gamma =  0.25  train duration time :  42.10186696052551
[['W' 'W' 'W' 'W' 'W' 'W' 'W' 'W' 'W' 'W' 'W']
 ['W' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'W']
 ['W' 'E' 'W' 'W' 'W' 'E' 'W' 'W' 'W' 'E' 'W']
 ['W' 'E' 'W' 'E' 'E' 'E' 'E' 'E' 'W' 'E' 'W']
 ['W' 'E' 'E' 'E' 'W' 'E' 'W' 'E' 'A' 'E' 'W']
 ['W' 'E' 'W' 'E' 'W' 'E' 'W' 'E' 'W' 'E' 'W']
 ['W' 'E' 'W' 'E' 'E' 'W' 'E' 'E' 'W' 'E' 'W']
 ['W' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'W']
 ['W' 'W' 'W' 'W' 'W' 'W' 'W' 'W' 'W' 'W' 'W']]
for gamma =  0.25  test duration time :  0.010433197021484375
done
for gamma =  0.5  train duration time :  48.255850315093994
[['W' 'W' 'W' 'W' 'W' 'W' 'W' 'W' 'W' 'W' 'W']
 ['W' 'E' 'A' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'W']
 ['W' 'E' 'W' 'W' 'W' 'E' 'W' 'W' 'W' 'E' 'W']
 ['W' 'E' 'W' 'E' 'E' 'E' 'E' 'E' 'W' 'E' 'W']
 ['W' 'E' 'E' 'E' 'W' 'E' 'W' 'E' 'E' 'E' 'W']
 ['W' 'E' 'W' 'E' 'W' 'E' 'W' 'E' 'W' 'E' 'W']
 ['W' 'E' 'W' 'E' 'E' 'W' 'E' 'E' 'W' 'E' 'W']
 ['W' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'W']
 ['W' 'W

gamma is a value of the future reward and as you can see for this problem when we increase the gamma the impact of next states will remove and the current state reward will be our reward, so we don't learn the match and we will just faced to overfit.


In [9]:
for alpha in [0.99, 0.6, 0.1]:
    packMan = PackMan()
    packMan.setAndChangeParams(alpha, packMan.gamma, 0.9)
    start = time.time()
    packMan.train()
    end = time.time()
    print("for alpha = " , alpha , " train duration time : ", end - start)
    start = time.time()
    packMan.test(False)
    end = time.time()
    print("for alpha = " , alpha , " test duration time : ", end - start)

done
for alpha =  0.99  train duration time :  58.32253313064575
[['W' 'W' 'W' 'W' 'W' 'W' 'W' 'W' 'W' 'W' 'W']
 ['W' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'W']
 ['W' 'E' 'W' 'W' 'W' 'E' 'W' 'W' 'W' 'E' 'W']
 ['W' 'E' 'W' 'E' 'E' 'E' 'E' 'E' 'W' 'E' 'W']
 ['W' 'E' 'E' 'E' 'W' 'E' 'W' 'E' 'A' 'E' 'W']
 ['W' 'E' 'W' 'E' 'W' 'E' 'W' 'E' 'W' 'E' 'W']
 ['W' 'E' 'W' 'E' 'E' 'W' 'E' 'E' 'W' 'E' 'W']
 ['W' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'W']
 ['W' 'W' 'W' 'W' 'W' 'W' 'W' 'W' 'W' 'W' 'W']]
for alpha =  0.99  test duration time :  0.00392460823059082
done
for alpha =  0.6  train duration time :  48.06031537055969
[['W' 'W' 'W' 'W' 'W' 'W' 'W' 'W' 'W' 'W' 'W']
 ['W' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'W']
 ['W' 'E' 'W' 'W' 'W' 'E' 'W' 'W' 'W' 'E' 'W']
 ['W' 'E' 'W' 'E' 'E' 'E' 'E' 'E' 'W' 'E' 'W']
 ['W' 'E' 'A' 'E' 'W' 'E' 'W' 'E' 'E' 'E' 'W']
 ['W' 'E' 'W' 'E' 'W' 'E' 'W' 'E' 'W' 'E' 'W']
 ['W' 'E' 'W' 'E' 'E' 'W' 'E' 'E' 'W' 'E' 'W']
 ['W' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'W']
 ['W' 'W' 

alpha is the learning rate, as you can see when we decrease it to 0 it means q table values don't have many changes but if we increase it to 0.6 we can see this changes and if alpha converge to 1 we can see that learning will occur quickly and q is not very important.


#### 2.4


#### 2.5


In [12]:
# As the question wanted, i print my q table:
flag = False
for i in packMan.qTable:
    for j in i:
        if(j!=0):
            flag = True
    if flag : print(i)
    flag = False        

[-1.  0.  0. -1.]
[ 6.05169454  1.45824     1.45824    -0.91      ]
[-0.973       1.99371063  0.          7.70888568]
[-1.e+000  1.e-323 -1.e+000  1.e-323]
[ 3.07356  0.      -0.7      0.784  ]
[ 1.e-323  1.e-323 -1.e+000 -1.e+000]
[-1.e+000  1.e-323 -1.e+000 -1.e+000]
[-0.7     0.      0.      0.5649]
[ 2.          0.         -0.38185442  0.        ]
[1.4 0.  0.  0. ]
[ 0.          0.         -0.99039227 -1.        ]
[ 0.45763096  0.45763096 -0.79988343 -0.77541772]
[-1.          0.         -1.         -0.99594852]
[ 2.      1.0192  0.     -0.7   ]
[ 0.          0.          0.59700566 -0.07316744]
[ 0.   0.   0.  -0.7]
[ 1.8463517   1.96372064 -0.68798049  2.0032352 ]
[-0.7         0.         -0.99757     5.61377639]
[-0.7         0.         -0.7         5.87416923]
[ 6.7232      2.7056556  -0.12499556  0.        ]
[0.         0.         0.         1.98384401]
[ 1.84167687  2.83430315 -0.64757466  7.35501821]
[ 0.71833886  0.784      -0.81051967  7.22436737]
[0.         0.         0. 

#### 2.6


I will change the init board and run the code


In [11]:
newBoard = [
            ['W', 'W', 'W', 'W', 'W', 'W'],
            ['W', 'A', 'D', 'D', 'D', 'W'],
            ['W', 'D', 'W', 'W', 'W', 'W'],
            ['W', 'D', 'W', 'D', 'D', 'W'],
            ['W', 'D', 'D', 'D', 'W', 'W'],
            ['W', 'D', 'W', 'D', 'W', 'W'],
            ['W', 'W', 'W', 'W', 'W', 'W']
        ]

packMan.initilaBoard = newBoard
packMan.makeNewBoard()
start = time.time()
packMan.test(False)
end=time.time()
print(end-start)

[['W' 'W' 'W' 'W' 'W' 'W']
 ['W' 'E' 'E' 'E' 'A' 'W']
 ['W' 'E' 'W' 'W' 'W' 'W']
 ['W' 'E' 'W' 'E' 'E' 'W']
 ['W' 'E' 'E' 'E' 'W' 'W']
 ['W' 'E' 'W' 'E' 'W' 'W']
 ['W' 'W' 'W' 'W' 'W' 'W']]
0.0395503044128418


#### 2.7 (Optional)


As you saw in the test, when I give True to display parameter You can see animation of the game
